# Função de Hash

Para implementar o ECDSA, é necessário recorrer a uma função criptográfica de *hash*. Como tal, utilizou-se o algoritmo `SHA256` da biblioteca `hashlib` como exemplo de uma função de *hash*, tendo-se criado uma função auxiliar para simplificar a leitura.

In [1]:
import hashlib

def sha256(payload):
    return hashlib.sha256(payload).hexdigest()

# Curva Elíptica

Para este exemplo, recorrer-se-á à curva elíptica `P-256 = y2 = x3 −3x + b`. Para tal, são definidos um conjunto de parâmetros que serão utilizados na construção da curva:
+ Um primo `p`
+ A ordem da curva `n`.
+ Um valor `seed` que servirá para a inicialização da pseudoaleatoriedade na escolha de elementos da curva.
+ `c`
+ Um termo independente da curva `b`.
+ Um ponto base da curva `G`, separado em dois valores, a abcissa `Gx` e a ordenada `Gy`.

In [2]:
NIST['P-256'] =  {
    'p': 115792089210356248762697446949407573530086143415290314195533631308867097853951,
    'n': 115792089210356248762697446949407573529996955224135760342422259061068512044369 ,
    'seed' : 'c49d360886e704936a6678e1139d26b7819f7e90',
    'c': '7efba1662985be9403cb055c75d4f7e0ce8d84a9c5114abcaf3177680104fa0d',
    'b': '5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b',
    'Gx' : '6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296',
    'Gy' : '4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5'
}

USED_NONCES = []

# Classe que implementa o ECDSA

## Inicialização

Ao ser chamada, a classe ECDSA não recebe parâmetros, recorrendo exclusivamente ao carregamento da curva explicitada. Como tal, implementa-se uma função auxiliar `setup` que efetua os cálculos necessários das variáveis da classe, retornando apenas a informação indispensável. Deste modo, as variáveis da classe são:
+ A instância da curva elíptica `E`.
+ O quoficiente do tamanho da curva `q`.
+ A ordem da curva `N`.
+ O ponto base da curva `G`.

## Geração do par de chaves

Quando necessário, é possível gerar um novo par de chaves recorrendo à função `gen_key_pair`. Esta gera um valor pseudoaleatório `d ∈ [0, n - 1]` que é utilizado como chave privada do esquema. Com esta e o ponto base, deriva-se a chave pública `Q`, calculando um ponto da curva correspondente.

## Assinatura de mensagens

Utilizando uma chave privada ECDSA `d`, é possível assinar uma mensagem `m` para posterior envio, com o intuito de assegurar a identidade do assinante. A função `sign` da classe segue o seguinte processo:
1. É gerado o valor de *hash* da mensagem a assinar `m' = HASH(m)`
2. É gerado um valor `k ∈ [0, q - 1]` que não poderá ter sido utilizado anteriormente pela instância da classe `ECDSA`.
3. É calculado o par de valores `r` e `s` que constituem a assinatura através do valor `k`, do ponto base `G` e a ordem da curva `N`.
4. É armazenado o valor `k` para que não seja de novo utilizado, protegendo o esquema de ataques.

## Verificação de mensagens

Ao receber uma mensagem assinada `m`, a assinatura `(r,s)` pode ser verificada recorrendo à chave pública correspondente ao assinante. Para tal, é efetuado o seguinte processo:
1. É gerado o valor de *hash* da mensagem recebida `m' = HASH(m)`
2. É calculado o valor `w`, módulo do inverso de `s`.
3. São calculados os valores `ri` e `mi`, multiplicando `w` por `r` e por `m`, respetivamente.
4. É calculado o valor da assinatura da mensagem recebida `r' = mi*G + ri*G (mod N)`
5. Verifica-se que `r'` é igual a `r`. Se tal for, a assinatura recebida considera-se válida.

In [3]:
class ECDSA:
    def __init__(self):
        self.E, self.q, self.N, self.G = self.setup()
        
    def setup(self):
        c = NIST['P-256']
        p  = c['p'] 
        n  = c['n']
        b  = ZZ(c['b'],16)
        Gx = ZZ(c['Gx'],16)
        Gy = ZZ(c['Gy'],16)
        E = EllipticCurve(GF(p),[-3,b])
        q = E.order()
        N = GF(q) # Zq
        G = E((Gx,Gy))
        return E, q, N, G
    
    def gen_key_pair(self):
        d = ZZ(self.N.random_element()) # private key
        Q = d*self.G                    # public key
        return d, Q
    
    def sign(self, m, d):
        r = 0; s = 0; N = self.N; G = self.G; q = self.q
        m = ZZ(sha256(message),16)
        while r == s == 0:
            k = ZZ.random_element(1, q)
            if not k in USED_NONCES:
                r = N((k*G).xy()[0])
                s = N((m+d*r)/N(k))
            USED_NONCES.append(k)
        return r, s
    
    def verify(self, m, r, s, Q):
        m = ZZ(sha256(message),16)
        w = 1/s
        ri = ZZ(r*w)
        mi = ZZ(m*w)
        if r == self.N((mi*self.G + ri*Q).xy()[0]):
            print 'Valid signature'
        else:
            print 'Invalid signature'

# Exemplo de utilização

In [4]:
ecdsa = ECDSA()

# Generate key pair
private_key, public_key = ecdsa.gen_key_pair()

# Message
message = "exemplo de uma mensagem!"

# Sign
ephemeral_key, signature = ecdsa.sign(message, private_key)

# Verify
ecdsa.verify(message, ephemeral_key, signature, public_key)

Valid signature
